## PBL(1): Individual Project Work

1. https://agtechresearch.pythonanywhere.com/ 에 접속하여 회원가입해 주세요. (비밀번호는 단순하게 만드는 것을 권장. 예: 1234)
2. `username` 에 이메일 형식의 아이디를 기입해 주세요.
3. `password` 에 비밀번호를 기입해 주세요.

In [1]:
project = "housingprice"  # 수정하지 마세요
username = "g4312@naver.com"  # 회원가입 시 사용한 이메일아이디 (예시. abc@hello.com)
password = "1234"  # 비밀번호

리더보드 제출을 위한 기본 설정: 아래 코드를 실행해주세요.


In [2]:
import os
import urllib.request

if not os.path.exists("competition.py"):
    url = "https://raw.githubusercontent.com/agtechresearch/LectureAlgorithm/main/competition/competition.py"
    filename = "competition.py"
    urllib.request.urlretrieve(url, filename)

아래 코드를 실행하여 데이터를 다운로드 받습니다: 3개의 csv 파일이 data 폴더에 다운로드됨

 * dataset.csv: 과거 주택매매 데이터 -> 학습에 사용할 데이터셋
 * problem.csv: 현재 A사가 매매를 고려하고 있는 130건의 주택정보 -> ML 모델에 의하여 예측을 수행하여야 할 데이터셋
 * submission.csv: 리더보드 서버 제출을 위한 파일 형식


In [3]:
import competition

# 파일 다운로드
competition.download_competition_files(project)

100%|██████████| 141k/141k [00:00<00:00, 383kiB/s] 


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

# 경고 무시
warnings.filterwarnings("ignore")

# Data 경로 설정
DATA_DIR = "data"

In [5]:
# 학습에 사용할 과거 주택매매 data set 로드 (dataset.csv)
dataset = pd.read_csv(os.path.join(DATA_DIR, "dataset.csv"))

# problem set 로드 (problem.csv)
problemset = pd.read_csv(os.path.join(DATA_DIR, "problem.csv"))

In [6]:
# # 결측치가 있다면 GarageType은 NO Garage로, GarageYrBlt의 값은 0으로 대체
dataset["GarageType"].fillna("No Garage", inplace=True)
dataset["GarageYrBlt"].fillna(0, inplace=True)

dataset.isnull().sum()


Id              0
LotArea         0
Street          0
LotConfig       0
OverallQual     0
OverallCond     0
YearBuilt       0
YearRemodAdd    0
TotalBsmtSF     0
1stFlrSF        0
2ndFlrSF        0
GrLivArea       0
FullBath        0
HalfBath        0
BedroomAbvGr    0
KitchenAbvGr    0
TotRmsAbvGrd    0
Fireplaces      0
GarageType      0
GarageYrBlt     0
GarageCars      0
GarageArea      0
YrSold          0
SalePrice       0
dtype: int64

In [12]:
# dataset을 train set과 val set으로 나누기
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(dataset.drop("SalePrice", axis=1), dataset["SalePrice"], test_size=0.2, random_state=42)

In [13]:
# 범주형 변수들을 모두 one-hot encoding하는데 x_train에 하는걸 그대로 x_val에도 적용
x_train = pd.get_dummies(x_train)
x_val = pd.get_dummies(x_test)


In [14]:
# Random over-sampling 적용
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=42)
x_train, y_train = ros.fit_resample(x_train, y_train)



In [19]:
x_train.head()

,Id,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,TotalBsmtSF,1stFlrSF,2ndFlrSF,GrLivArea,...,LotConfig_FR2,LotConfig_FR3,LotConfig_Inside,GarageType_2Types,GarageType_Attchd,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,GarageType_Detchd,GarageType_No Garage
0,1149,13132,5,5,1914,1950,747,892,892,1784,...,False,False,True,False,False,False,False,False,True,False
1,808,9938,7,5,1993,1994,1050,1062,887,1949,...,False,False,True,False,True,False,False,False,False,False
2,1288,6000,6,7,1929,1950,862,950,208,1158,...,False,False,False,False,False,False,True,False,False,False
3,591,10452,6,5,1953,1953,1094,1094,0,1094,...,False,False,True,False,True,False,False,False,False,False
4,1189,11345,5,5,1948,1950,928,928,992,1920,...,False,False,False,False,False,False,False,False,True,False


오버샘플링을 하려고 했으나 소수 클래스에 대해 적은 수의 샘플이 있어서 SMOTE 또는 ADASYN이 적절한 이웃을 찾지 못하고 있다
그럼에도 불구하고 오버샘플링은 중요해보인다. 그렇다면 개수가 적은 샘플들은 다른 값으로 치환해 버릴까?

In [18]:
# Saleprice, Street, LotConfig, GarageType를 제외한 특성 선택


from sklearn.preprocessing import StandardScaler




# Saleprice, Street, LotConfig, GarageType를 제외한 특성 선택
features_to_scale = x_train.drop(columns=['Saleprice', 'Street', 'LotConfig', 'GarageType'])

# 스케일 변환 수행
scaler = StandardScaler()
scaled_features = scaler.fit_transform(features_to_scale)

# 변환된 데이터를 다시 데이터프레임으로 변환
scaled_data = pd.DataFrame(scaled_features, columns=features_to_scale.columns)

# 변환된 데이터와 Saleprice, Street, LotConfig, GarageType를 다시 합치기
final_data = pd.concat([scaled_data, x_train[['Saleprice', 'Street', 'LotConfig', 'GarageType']]], axis=1)
# x_test에 스케일 변환 적용
x_test_features = x_test.drop(columns=['Saleprice', 'Street', 'LotConfig', 'GarageType'])
scaled_x_test_features = scaler.transform(x_test_features)
scaled_x_test = pd.DataFrame(scaled_x_test_features, columns=x_test_features.columns)
x_test_final = pd.concat([scaled_x_test, x_test[['Saleprice', 'Street', 'LotConfig', 'GarageType']]], axis=1)

# x_problem에 스케일 변환 적용
x_problem_features = x_problem.drop(columns=['Saleprice', 'Street', 'LotConfig', 'GarageType'])
scaled_x_problem_features = scaler.transform(x_problem_features)
scaled_x_problem = pd.DataFrame(scaled_x_problem_features, columns=x_problem_features.columns)
x_problem_final = pd.concat([scaled_x_problem, x_problem[['Saleprice', 'Street', 'LotConfig', 'GarageType']]], axis=1)




KeyError: "['Saleprice', 'Street', 'LotConfig', 'GarageType'] not found in axis"

ValueError: The feature names should match those that were passed during fit.
Feature names seen at fit time, yet now missing:
- LotConfig_FR3


In [ ]:
# lightgbm 모델 학습
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
lgb = lgb.LGBMRegressor()
lgb.fit(x_train_preprocessed, y_train)



[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000525 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2110
[LightGBM] [Info] Number of data points in the train set: 8720, number of used features: 20
[LightGBM] [Info] Start training from score 195363.880734


LGBMRegressor()

In [ ]:
from sklearn.metrics import mean_squared_error

train_pred = lgb.predict(x_train_preprocessed)
test_pred = lgb.predict(x_test_preprocessed)

print("Train RMSE :", mean_squared_error(y_train, train_pred) ** 0.5)
print("Test RMSE :", mean_squared_error(y_val, test_pred) ** 0.5)

Train RMSE : 5500.783640852231
Test RMSE : 31024.262137087346


In [ ]:
problem_pred = lgb.predict(x_problem_preprocessed)

In [ ]:
# 리더보드 서버 제출을 위한 파일 생성
submission = pd.read_csv(os.path.join(DATA_DIR, "submission.csv"))
submission["SalePrice"] = problem_pred

# 예측 결과 화면에 출력 후 제출
display(submission)
competition.submit(project, username, password, submission)

,Id,SalePrice
0,1341,133706.919249
1,1342,138311.563329
2,1343,159309.992832
3,1344,136393.401806
4,1345,95013.403699
...,...,...
125,1466,286122.110394
126,1467,171638.503779
127,1468,76867.829523
128,1469,98008.392176


아이디:  g4312@naver.com
파일명:  submissions\20240514-142521-submission.csv
[제출에 성공하였습니다]
제출 결과: 25182.983355057535
